In [1]:
import pickle
import json
from dataclasses import dataclass
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
import argparse

In [2]:
# Embedding and Position
class EmbeddingToken(nn.Module):
    """
    embedding for token and position, B*T --> B*T*C
    """
    
    def __init__(self, n_embd, voc_size, block_size, weight=None):
        super().__init__()
        self.block_size = block_size
        self.wte = nn.Embedding(voc_size, n_embd)
        self.wpe = nn.Embedding(block_size, n_embd)
        if weight is not None:
            self.wte.weight = weight
        
    def forward(self, x):
        positions = torch.arange(self.block_size, device=x.device).unsqueeze(0).repeat(x.shape[0], 1)
        return self.wte(x) + self.wpe(positions)

In [3]:
# Attention
class SelfAttention(nn.Module):
    
    def __init__(self, n_embd, heads, dropout, bias, pad_val):
        super().__init__()
        assert n_embd % heads == 0
        
        # reflect the n input into 3n size for q,k,v
        self.attn = nn.Linear(n_embd, 3*n_embd, bias = bias)
        # output projection
        self.proj = nn.Linear(n_embd, n_embd, bias = bias)

        self.resid_dropout = nn.Dropout(dropout)
        self.n_embd = n_embd
        self.heads = heads
        self.dropout = dropout
        self.pad_val = pad_val
        
        # GPU accelaration if possible
        self.flash = hasattr(torch.nn.functional, "scaled_dot_product_attention")
        if not self.flash:
            raise AttributeError("torch version does not support gpu attention.")
            
    def forward(self, x):
        B, T, C = x.size()
        
        # get q,k,v
        Q, K, V = self.attn(x).split(self.n_embd, dim=2)
        
        # split into headers
        # B*T*C --> B*T*N*S --> B*N*T*S (N is heads, S is heads' feature dim)
        q = Q.view(B, T, self.heads, C // self.heads).transpose(
            1, 2
        )
        k = K.view(B, T, self.heads, C // self.heads).transpose(
            1, 2
        )
        v = V.view(B, T, self.heads, C // self.heads).transpose(
            1, 2
        )
        
        # attention calculation
        y = torch.nn.functional.scaled_dot_product_attention(
            q,
            k,
            v,
            attn_mask=None,
            dropout_p=self.dropout if self.training else 0,
            is_causal=True,
        )
        
        # concat multiheads back to one
        # B*N*T*S --> B*T*N*S --> B*T*C
        y = y.transpose(1, 2).contiguous().view(B,T,C)
        
        # output projection
        y = self.resid_dropout(self.proj(y))
        
        return y, K, V

class CrossAttention(SelfAttention):
    
    def __init__(self, n_embd, heads, dropout, bias, pad_val):
        super().__init__(n_embd, heads, dropout, bias, pad_val)
        self.attn = nn.Linear(n_embd, n_embd, bias=bias)
    
    def forward(self, x, K, V):
        B, T, C = (x.size())
        
        Q = self.attn(x)
        # split into headers
        # B*T*C --> B*T*N*S --> B*N*T*S (N is heads, S is heads' feature dim)
        q = Q.view(B, T, self.heads, C // self.heads).transpose(
            1, 2
        )
        k = K.view(B, T, self.heads, C // self.heads).transpose(
            1, 2
        )
        v = V.view(B, T, self.heads, C // self.heads).transpose(
            1, 2
        )
        
        # attention calculation
        y = torch.nn.functional.scaled_dot_product_attention(
            q,
            k,
            v,
            attn_mask=None,
            dropout_p=self.dropout if self.training else 0,
            is_causal=True,
        )
        
        # concat multiheads back to one
        # B*N*T*S --> B*T*N*S --> B*T*C
        y = y.transpose(1, 2).contiguous().view(B,T,C)
        
        # output projection
        y = self.resid_dropout(self.proj(y))
        
        return y

In [4]:
class Normalization(nn.Module):
    
    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None
        
    def forward(self, x):
        return F.layer_norm(x, self.weight.shape, self.weight, self.bias, 1e-5)

In [5]:
class FeedForward(nn.Module):
    
    def __init__(self, n_embd, dropout, bias):
        super().__init__()
        self.dim_inc = nn.Linear(n_embd, 4*n_embd, bias=bias)
        self.gelu = nn.GELU()
        self.dim_red = nn.Linear(4*n_embd, n_embd, bias=bias)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = self.dim_inc(x)
        x = self.gelu(x)
        x = self.dim_red(x)
        x = self.dropout(x)
        return x

In [6]:
# Encoder Block
class EncoderBlock(nn.Module):
    
    def __init__(self, config):
        super().__init__()
        
        self.attn = SelfAttention(
            n_embd=config.n_embd, 
            heads=config.heads,
            dropout=config.dropout,
            bias=config.bias,
            pad_val=config.pad_val
        )
        self.ff = FeedForward(
            n_embd=config.n_embd,
            dropout=config.dropout,
            bias=config.bias,
        )
        self.add_and_norm = Normalization(
            ndim=config.ndim, 
            bias=config.bias
        )
        
    def forward(self, x, needkv=False):
        # self attention
        y, K, V = self.attn(x)
        # add and norm
        x = self.add_and_norm(x+y)
        # feedforward
        z = self.ff(x)
        # add and norm
        res = self.add_and_norm(x+z)
        
        if (needkv):
            return res, K, V
        else:
            return res

In [7]:
# Decoder Block
class DecoderBlock(nn.Module):

    def __init__(self, config):
        super().__init__()
        
        self.selfattn = SelfAttention(
            n_embd=config.n_embd, 
            heads=config.heads,
            dropout=config.dropout,
            bias=config.bias,
            pad_val=config.pad_val,
        )
        self.crossattn = CrossAttention(
            n_embd=config.n_embd, 
            heads=config.heads,
            dropout=config.dropout,
            bias=config.bias,
            pad_val=config.pad_val,
        )
        self.ff = FeedForward(
            n_embd=config.n_embd,
            dropout=config.dropout,
            bias=config.bias,
        )
        self.add_and_norm = Normalization(
            ndim=config.ndim, 
            bias=config.bias
        )
        
    def forward(self, x, K, V):
        # self attenion
        y, K_notuse, V_notuse = self.selfattn(x)
        z = self.add_and_norm(x+y)
        # cross attention
        m = self.crossattn(z, K, V)
        m = self.add_and_norm(m+z)
        # feedforward
        n = self.ff(m)
        res = self.add_and_norm(m+n)
        
        return res

In [8]:
class Predict(nn.Module):
    
    def __init__(self, n_embd, voc_size, bias):
        super().__init__()
        # Linear layer (n_embd --> voc_size)
        self.transform = nn.Linear(n_embd, voc_size, bias=bias)
        
    def forward(self, x):
        x = self.transform(x)
        # get prediction
        x = F.softmax(x, dim=-1)
        
        return x

In [9]:
def get_loss(pred_res, target):
    loss_fn = nn.CrossEntropyLoss()
    # Reshape predictions and target to the correct dimensions
    pred_res = pred_res.reshape(-1, pred_res.size(-1))  # (batch_size * seq_len, vocab_size)
    target = target.reshape(-1).long()  # (batch_size * seq_len)
    # pred_res = torch.tensor(pred_res)
    # target = torch.tensor(target)
    return loss_fn(pred_res, target)


In [10]:
class Transformer(nn.Module):
    
    def __init__(self, config):
        super().__init__()
        
        # embedding layer: B*T --> B*T*C
        self.embedding = EmbeddingToken(
            n_embd=config.n_embd,
            voc_size=config.voc_size,
            block_size=config.block_size,
            weight=None,
        )
        
        # encoder block: 
        # needkv = False: B*T*C --> result (B*T*C)
        # needkv = True: B*T*C --> result & K & V (B*T*C)
        self.encoders = nn.ModuleList(
            [EncoderBlock(config=config) for _ in range(config.encoder_iter)]
        )
        
        # decoder block: B*T*C , K , V --> result (B*T*C)
        self.decoders = nn.ModuleList(
            [DecoderBlock(config=config) for _ in range(config.decoder_iter)]
        )
        
        # prediction layer: B*T*C --> B*T*voc
        self.pred = Predict(
            n_embd=config.n_embd, 
            voc_size=config.voc_size, 
            bias=config.bias
        )
        
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.xavier_normal_(module.weight)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.1)
            
    def forward(self, idx):
        # the iput should be [[article data: list of [sentence]], [summary data], [target data]] shape
        einput = idx[0]
        dinput = idx[1]
        target = idx[2]
        
        # embedding
        einput = self.embedding(einput)
        dinput = self.embedding(dinput)
        
        # encoder
        for i,encoder in enumerate(self.encoders):
            if i == len(self.encoders) - 1:
                eoutput, K, V = encoder(einput, needkv=True)
            else:
                einput = encoder(einput, needkv=False)
        
        # decoder
        for i,decoder in enumerate(self.decoders):
            if i == len(self.decoders) - 1:
                doutput = decoder(dinput, K=K, V=V)
            else:
                dinput = decoder(dinput, K=K, V=V)
        
        # prediction
        pred_res = self.pred(doutput)
        
        return pred_res

In [11]:
@dataclass
class TransformerConfig:
    encoder_iter: int = 6
    decoder_iter: int = 6
    block_size: int = 512
    voc_size: int = (
        132574 + 132574 % 64
    )
    ndim: int = 512
    heads: int = 8
    n_embd: int = 512
    dropout: float = 0.0
    bias: bool = (
        True  
    )
    pad_val: int = 16175

In [12]:
# TESTING
# config = TransformerConfig
# tryinit = Transformer(config=config)
# test_data = np.random.randint(size=(5,6,512), low=0, high=100)

In [13]:
def load_data(path):
    
    if path.endswith('.npy'):
        data = np.load(path)
        
    elif path.endswith('.csv'):
        pass
    elif path.endswith('.pkl'):
        with open(path, 'rb') as file:
            data = pickle.load(file)
    else:
        raise TypeError(f'{path[:-3]} format not supported.')
    
    return data

In [14]:
from torch.utils.data import Dataset
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [15]:
import torch

def train(batch_size, lr, config, num_epochs, summary=False, datas=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if datas is not None:
        train_data = datas[0]
        val_data = datas[1]
    else:
        train_data = load_data('C:/Users/Nick/Desktop/NLP/data/CNN/train.npy')
        val_data = load_data('C:/Users/Nick/Desktop/NLP/data/CNN/val.npy')

    model = Transformer(config=config).to(device)
    model.train()

    # Adam
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # dataloader
    train_loader = torch.utils.data.DataLoader(MyDataset(train_data.transpose(1, 0, 2)), batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(MyDataset(val_data.transpose(1, 0, 2)), batch_size=batch_size, shuffle=True)
    
    # tensorboard
    if summary:
        from torch.utils.tensorboard import SummaryWriter
        writer = SummaryWriter(log_dir='logs')
    best_val_loss = float('inf')
    
    train_loss = 0
    
    # train
    for epoch in range(num_epochs):
        total_loss = 0.0

        for i,batch in enumerate(train_loader):
            if i % 10 == 0:
                print(i/len(train_loader))
            einputs, dinputs, targets = batch.permute(1, 0, 2)

            einputs = einputs.to(device)
            dinputs = dinputs.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()

            # go through model
            predictions = model([einputs, dinputs, targets])

            loss = get_loss(predictions, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            # clean memo
            del einputs, dinputs, targets, predictions, loss
            torch.cuda.empty_cache()

        train_loss = avg_loss = total_loss / len(train_loader)
        if summary:
            writer.add_scalar('Training Loss', avg_loss, epoch) 

        # val
        val_loss = validate(model, val_loader, device)
        if summary:
            writer.add_scalar('Validation Loss', val_loss, epoch)

        # store best checkpoints
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'checkpoints/best_model.pth')
            print(f'Best model saved at epoch {epoch + 1} with validation loss: {val_loss:.4f}')

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}, Validation Loss: {val_loss:.4f}')

    # save the last model
    torch.save(model.state_dict(), 'checkpoints/last_model.pth')
    print('Last model saved.')

    if summary:
        writer.close()
        
    # clean memo
    del model
    del optimizer
    del train_loader
    del val_loader
    torch.cuda.empty_cache()

    return train_loss, val_loss
    
def validate(model, val_data, device):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch in val_data:
            einputs, dinputs, targets = batch.permute(1, 0, 2)
            einputs, dinputs, targets = einputs.to(device), dinputs.to(device), targets.to(device)

            predictions = model([einputs, dinputs, targets])
            loss = get_loss(predictions, targets)
            total_loss += loss.item()

            # clean memo
            del einputs, dinputs, targets, predictions, loss
            torch.cuda.empty_cache()

    avg_val_loss = total_loss / len(val_data)
    print(f'Validation Loss: {avg_val_loss:.4f}')
    return avg_val_loss

In [16]:
if __name__ == "main":
    out_dir = "C:/Users/Nick/Desktop/NLP/results/transformer/result_base.json"
    meta_data = load_data("C:/Users/Nick/Desktop/NLP/data/CNN/meta.pkl")
    voc_size = meta_data['voc_size']
    config = TransformerConfig()
    config.voc_size = voc_size + voc_size % 64
    torch.cuda.empty_cache()
    train_loss, val_loss = train(batch_size=4, lr=0.01, config=config, num_epochs=10, summary=False, datas=None)
    
    to_store = {
        "train_loss" : train_loss,
        "val_loss" : val_loss,
    }
    with open("C:/Users/Nick/Desktop/NLP/results/transformer/result.json") as fp:
        json.dump(to_store, fp)